# Environment Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'spouse',
    'postgres': False,
    'parallelism': 1,
    'db_name': 'babble_spouse_demo',
    'babbler_candidate_split': 1,
    'babbler_label_split': 1,
    'supervision': 'generative',
    'disc_model_class': 'logreg',
    'gen_model_search_space': 1,
    'disc_model_search_space': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse_demo.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs
config = merge_configs(config)

Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting disc_model_search_space=10 to disc_model_search_space=1
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting gen_model_search_space=10 to gen_model_search_space=1
Overwriting disc_model_class=lstm to disc_model_class=logreg


In [5]:
from snorkel.models import candidate_subclass
from tutorials.babble.spouse import SpousePipeline

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
candidate_class = Spouse
pipe = SpousePipeline(session, Spouse, config)

### Initialization

In [6]:
# %time pipe.parse()

In [7]:
# %time pipe.extract()

In [8]:
# %time pipe.load_gold()

In [9]:
# %time pipe.featurize()

### Pre-load Explanations (10) + User Lists (4)

In [10]:
from snorkel.contrib.babble import BabbleStream
bs = BabbleStream(session, candidate_class=Spouse, balanced=True, shuffled=True, seed=1234)

In [11]:
from tutorials.babble.spouse.spouse_examples import get_explanations, get_user_lists

# spouse_explanations = get_explanations()
# spouse_user_lists = get_user_lists()
spouse_explanations = []
spouse_user_lists = {}

In [12]:
# bs.preload(explanations=spouse_explanations, user_lists=spouse_user_lists)

# Start Demo:

### View a candidate

### Give an explanation

In [13]:
candidates = session.query(bs.candidate_class).filter(bs.candidate_class.split == 1).all()

In [26]:
LABEL = False
CONDITION = "The word 'son' is immediately to the left of Y"

With the user input and the current candidate, we make an Explanation object.

In [27]:
from snorkel.contrib.babble import Explanation
explanation = Explanation(CONDITION, LABEL, candidate=None)

### Parse and view labeling stats

In [28]:
%time parse_list, filtered_parses, conf_matrix_list, stats_list = bs.apply(explanation)

Flushing all parses from previous explanation set.
1 explanation(s) out of 1 were parseable.
1 parse(s) generated from 1 explanation(s).
1 parse(s) remain (0 parse(s) removed by DuplicateSemanticsFilter).
Note: 1 LFs did not have candidates and therefore could not be filtered.
1 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
### Applying labeling functions to split 1
[========================================] 100%

### Done in 0.1s.

1 parse(s) remain (0 parse(s) removed by UniformSignatureFilter: (0 None, 0 All)).
1 parse(s) remain (0 parse(s) removed by DuplicateSignatureFilter).
CPU times: user 272 ms, sys: 11.4 ms, total: 283 ms
Wall time: 279 ms


In [29]:
PARSE_IDX = 0
if parse_list:
    parse = parse_list[PARSE_IDX]
    conf_matrix = conf_matrix_list[PARSE_IDX]
    stats = stats_list[PARSE_IDX]

    print("Parse {}:\n{}\n".format(PARSE_IDX, bs.semparser.grammar.translate(parse.semantics)))
    print(stats.accuracy)
    print(stats.class_coverage)
    print(stats.coverage)
else:
    print("No valid parses were found.")

Parse 0:
return -1 if 'son'.in(text(exactly 1 word(s) to the left of Y)) else 0

Accuracy: 100.00% (21/21)
ClassCoverage: 0.81% (21/2600)
Coverage: 0.75% (21/2796)


In [20]:
bs.filtered_analysis(filtered_parses)

No filtered parses to analyze.


In [21]:
# bs.semparser.grammar.print_chart()

In [22]:
lf = parse_list[0].function

In [23]:
matches = []
for c in candidates:
    if lf(c):
        matches.append(c)

In [24]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(matches[:100], session, n_per_page=3, height=300)
sv

<IPython.core.display.Javascript object>

In [31]:
c = sv.get_selected()
print(c.get_stable_id())

7fc3e510-c4e6-44c2-a24b-f9a39bfcfb07::span:4942:4950~~7fc3e510-c4e6-44c2-a24b-f9a39bfcfb07::span:4973:4978


## SCRATCH

### View labeled candidates

Select the subset of labeled candidates you would like to view.

In [ ]:
SUBSET = 'correct' # Must be one of ['correct', 'incorrect', 'abstained']

subset = getattr(conf_matrix_list[0], SUBSET)
def candidate_generator(subset):
    for c in subset: 
        yield c
subset_generator = candidate_generator(subset)

In [ ]:
from snorkel.viewer import SentenceNgramViewer
c = subset_generator.next()
sv = SentenceNgramViewer([c], session, n_per_page=3, height=max(len(c.get_parent().words)*2, 80))
sv

### Commit parses

If you are satisfied with the given parses, commit them.

In [ ]:
bs.commit()

In [ ]:
bs.get_majority_quality(split=2)

In [ ]:
# %time bs.label_split(0)

In [ ]:
# %time bs.label_split(2)

In [ ]:
# This is here for illustration purposes.
# No need to call this every time; it will be called by set_babbler_matrices.
%time bs.get_label_matrix(0)

### View global stats

In [ ]:
bs.get_global_coverage()

In [ ]:
bs.get_lf_stats()

In [ ]:
bs.get_parses()

In [ ]:
bs.get_lfs()

In [ ]:
bs.get_explanations()

## REPEAT (go back to "START DEMO")

When you have entered all of the explanations that you would like to, run these final cells.

In [ ]:
# NOTE: Don't use this. Instead, incrementally label using bs.label_split() after each commit.
# pipe.lfs = bs.get_lfs()
# %time pipe.label()

In [ ]:
# %time pipe.set_babbler_matrices(bs) # Pulls out and saves label matrices from babbler.

In [ ]:
# %time pipe.supervise()

In [ ]:
# %time pipe.classify()

Note: in general, we expect Disc to do better than Gen. However, with small sample sizes, major class imbalance, or lack of grid search, those may flip.